IMPORTAÇÃO DE BIBLIOTECAS NECESSÁRIAS

In [197]:
import pandas as pd
import kagglehub

import pycountry
import pycountry_convert as pc

EXTRAÇÃO DOS DADOS DO KAGGLE

In [198]:
# Download da versão mais recente
path = kagglehub.dataset_download("anshtanwar/global-data-on-sustainable-energy")

print("Path to dataset files:", path)

df = pd.read_csv('data/global-data-on-sustainable-energy (1).csv')


Path to dataset files: C:\Users\Paula\.cache\kagglehub\datasets\anshtanwar\global-data-on-sustainable-energy\versions\7


TRANSFORMAÇÕES

In [199]:

df.head()

,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),...,Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,Afghanistan,2000,1.613591,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,...,302.59482,1.64,760.000000,NaN,NaN,NaN,60,652230.0,33.93911,67.709953
1,Afghanistan,2001,4.074574,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,...,236.89185,1.74,730.000000,NaN,NaN,NaN,60,652230.0,33.93911,67.709953
2,Afghanistan,2002,9.409158,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,...,210.86215,1.40,1029.999971,NaN,NaN,179.426579,60,652230.0,33.93911,67.709953
3,Afghanistan,2003,14.738506,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,...,229.96822,1.40,1220.000029,NaN,8.832278,190.683814,60,652230.0,33.93911,67.709953
4,Afghanistan,2004,20.064968,10.9,7.75,NaN,44.24,0.33,0.0,0.56,...,204.23125,1.20,1029.999971,NaN,1.414118,211.382074,60,652230.0,33.93911,67.709953


In [200]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3649 entries, 0 to 3648
Data columns (total 21 columns):
 #   Column                                                            Non-Null Count  Dtype  
---  ------                                                            --------------  -----  
 0   Entity                                                            3649 non-null   object 
 1   Year                                                              3649 non-null   int64  
 2   Access to electricity (% of population)                           3639 non-null   float64
 3   Access to clean fuels for cooking                                 3480 non-null   float64
 4   Renewable-electricity-generating-capacity-per-capita              2718 non-null   float64
 5   Financial flows to developing countries (US $)                    1560 non-null   float64
 6   Renewable energy share in the total final energy consumption (%)  3455 non-null   float64
 7   Electricity from fossil fuels (TW

In [201]:
df.columns

Index(['Entity', 'Year', 'Access to electricity (% of population)',
       'Access to clean fuels for cooking',
       'Renewable-electricity-generating-capacity-per-capita',
       'Financial flows to developing countries (US $)',
       'Renewable energy share in the total final energy consumption (%)',
       'Electricity from fossil fuels (TWh)', 'Electricity from nuclear (TWh)',
       'Electricity from renewables (TWh)',
       'Low-carbon electricity (% electricity)',
       'Primary energy consumption per capita (kWh/person)',
       'Energy intensity level of primary energy (MJ/$2017 PPP GDP)',
       'Value_co2_emissions_kt_by_country',
       'Renewables (% equivalent primary energy)', 'gdp_growth',
       'gdp_per_capita', 'Density\n(P/Km2)', 'Land Area(Km2)', 'Latitude',
       'Longitude'],
      dtype='object')

In [202]:
# Renomeando colunas para facilitar compreensão e manipulação

renomeia_colunas = {'Density\\n(P/Km2)': 'PopulationalDensity',
                    'Energy intensity level of primary energy (MJ/$2017 PPP GDP)':'energy_intensity',
                    'Low-carbon electricity (% electricity)':'low_carbon_generation',
                    'Value_co2_emissions_kt_by_country': 'co2_emissions_kt',
                    'Renewables (% equivalent primary energy)' : 'renewable_energy',
                    'Land Area(Km2)':'landArea',
                    'Access to electricity (% of population)':'access_electricity',
                    'Renewable-electricity-generating-capacity-per-capita':'renewable_generating_capacity_per_capita',
                    'Financial flows to developing countries (US $)':'financial_flows_developing_countries',
                    'Renewable energy share in the total final energy consumption (%)':'renewable_share_final_energy_consumption',
                    'Access to clean fuels for cooking':'access_clean_fuels_cooking',
                    'Electricity from fossil fuels (TWh)':'electricity_fossil_fuels',
                    }
df.rename(columns=renomeia_colunas, inplace=True)

In [203]:
# Segmentação do Dataframe em 'Países' e 'Dados de energia'

paises = df[['Entity','Latitude','Longitude','landArea','PopulationalDensity']]
paises = paises.drop_duplicates().reset_index(drop=True)
paises['id_pais'] = range(1, len(paises) + 1)

In [204]:
# Mapeamento de países por continente
def get_continent(country_name):
    try:
        # Corrige nomes que não estão exatos
        country = pycountry.countries.lookup(country_name)
        country_alpha2 = country.alpha_2
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except Exception as e:
        return 'Desconhecido'  # País não encontrado

# Cria a coluna de continente
paises['Continente'] = paises['Entity'].apply(get_continent)


In [205]:
# Cria a coluna Signatario como True/False com base na lista
nao_signatarios = ['Libya', 'Iran', 'Yemen', 'United States']
paises['Signatario'] = ~paises['Entity'].isin(nao_signatarios)

In [206]:
# Criação da Chave de continentes para normalização
continentes = {
    'id_continente': [1, 2, 3, 4, 5, 6, 7],
    'Continente': ['Africa', 'North America', 'South America', 'Asia', 'Europe', 'Oceania', 'Antartica']
}

continentes = pd.DataFrame(continentes)
continentes

,id_continente,Continente
0,1,Africa
1,2,North America
2,3,South America
3,4,Asia
4,5,Europe
5,6,Oceania
6,7,Antartica


In [207]:
# Substituição da dimensão continente por sua identificação
paises = paises.merge(continentes, on='Continente', how='left')
paises = paises.drop(columns=['Continente'])
paises['id_continente'] = paises['id_continente'].fillna(0).astype(int)

In [208]:
paises

,Entity,Latitude,Longitude,landArea,PopulationalDensity,id_pais,Signatario,id_continente
0,Afghanistan,33.939110,67.709953,652230.0,60,1,True,4
1,Albania,41.153332,20.168331,28748.0,105,2,True,5
2,Algeria,28.033886,1.659626,2381741.0,18,3,True,1
3,Angola,-11.202692,17.873887,1246700.0,26,4,True,1
4,Antigua and Barbuda,17.060816,-61.796428,443.0,223,5,True,2
...,...,...,...,...,...,...,...,...
171,Uzbekistan,41.377491,64.585262,447400.0,79,172,True,4
172,Vanuatu,-15.376706,166.959158,12189.0,25,173,True,6
173,Yemen,15.552727,48.516388,527968.0,56,174,False,4
174,Zambia,-13.133897,27.849332,752618.0,25,175,True,1


In [209]:
dados = df.drop(columns=['landArea', 'Latitude','Longitude','PopulationalDensity'])
dados

,Entity,Year,access_electricity,access_clean_fuels_cooking,renewable_generating_capacity_per_capita,financial_flows_developing_countries,renewable_share_final_energy_consumption,electricity_fossil_fuels,Electricity from nuclear (TWh),Electricity from renewables (TWh),low_carbon_generation,Primary energy consumption per capita (kWh/person),energy_intensity,co2_emissions_kt,renewable_energy,gdp_growth,gdp_per_capita
0,Afghanistan,2000,1.613591,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,65.957440,302.59482,1.64,760.000000,NaN,NaN,NaN
1,Afghanistan,2001,4.074574,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,84.745766,236.89185,1.74,730.000000,NaN,NaN,NaN
2,Afghanistan,2002,9.409158,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,81.159424,210.86215,1.40,1029.999971,NaN,NaN,179.426579
3,Afghanistan,2003,14.738506,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,67.021280,229.96822,1.40,1220.000029,NaN,8.832278,190.683814
4,Afghanistan,2004,20.064968,10.9,7.75,NaN,44.24,0.33,0.0,0.56,62.921350,204.23125,1.20,1029.999971,NaN,1.414118,211.382074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,Zimbabwe,2016,42.561730,29.8,62.88,30000.0,81.90,3.50,0.0,3.32,48.680350,3227.68020,10.00,11020.000460,NaN,0.755869,1464.588957
3645,Zimbabwe,2017,44.178635,29.8,62.33,5570000.0,82.46,3.05,0.0,4.30,58.503407,3068.01150,9.51,10340.000150,NaN,4.709492,1235.189032
3646,Zimbabwe,2018,45.572647,29.9,82.53,10000.0,80.23,3.73,0.0,5.46,59.412407,3441.98580,9.83,12380.000110,NaN,4.824211,1254.642265
3647,Zimbabwe,2019,46.781475,30.1,81.40,250000.0,81.50,3.66,0.0,4.58,55.582527,3003.65530,10.47,11760.000230,NaN,-6.144236,1316.740657


In [210]:
# Substituição da dimensão país pelo identificador
dados = dados.merge(paises[['Entity', 'id_pais']], on='Entity', how='left')
dados = dados.drop(columns='Entity')


In [211]:
# Padronização de percentuais
dados[['access_electricity']] = dados[['access_electricity']].round(4)
dados[['low_carbon_generation']] = dados[['low_carbon_generation']].round(4)
dados[['renewable_share_final_energy_consumption']] = dados[['renewable_share_final_energy_consumption']].round(4)
dados[['renewable_energy']] = dados[['renewable_energy']].round(4)

In [212]:
dados

,Year,access_electricity,access_clean_fuels_cooking,renewable_generating_capacity_per_capita,financial_flows_developing_countries,renewable_share_final_energy_consumption,electricity_fossil_fuels,Electricity from nuclear (TWh),Electricity from renewables (TWh),low_carbon_generation,Primary energy consumption per capita (kWh/person),energy_intensity,co2_emissions_kt,renewable_energy,gdp_growth,gdp_per_capita,id_pais
0,2000,1.6136,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,65.9574,302.59482,1.64,760.000000,NaN,NaN,NaN,1
1,2001,4.0746,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,84.7458,236.89185,1.74,730.000000,NaN,NaN,NaN,1
2,2002,9.4092,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,81.1594,210.86215,1.40,1029.999971,NaN,NaN,179.426579,1
3,2003,14.7385,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,67.0213,229.96822,1.40,1220.000029,NaN,8.832278,190.683814,1
4,2004,20.0650,10.9,7.75,NaN,44.24,0.33,0.0,0.56,62.9214,204.23125,1.20,1029.999971,NaN,1.414118,211.382074,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,2016,42.5617,29.8,62.88,30000.0,81.90,3.50,0.0,3.32,48.6804,3227.68020,10.00,11020.000460,NaN,0.755869,1464.588957,176
3645,2017,44.1786,29.8,62.33,5570000.0,82.46,3.05,0.0,4.30,58.5034,3068.01150,9.51,10340.000150,NaN,4.709492,1235.189032,176
3646,2018,45.5726,29.9,82.53,10000.0,80.23,3.73,0.0,5.46,59.4124,3441.98580,9.83,12380.000110,NaN,4.824211,1254.642265,176
3647,2019,46.7815,30.1,81.40,250000.0,81.50,3.66,0.0,4.58,55.5825,3003.65530,10.47,11760.000230,NaN,-6.144236,1316.740657,176


In [213]:
dados.to_csv("data/dados.csv", index=False)

In [214]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3649 entries, 0 to 3648
Data columns (total 17 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Year                                                3649 non-null   int64  
 1   access_electricity                                  3639 non-null   float64
 2   access_clean_fuels_cooking                          3480 non-null   float64
 3   renewable_generating_capacity_per_capita            2718 non-null   float64
 4   financial_flows_developing_countries                1560 non-null   float64
 5   renewable_share_final_energy_consumption            3455 non-null   float64
 6   electricity_fossil_fuels                            3628 non-null   float64
 7   Electricity from nuclear (TWh)                      3523 non-null   float64
 8   Electricity from renewables (TWh)                   3628 non-null   float64
 9

In [215]:
paises.to_csv("data/paises.csv", index=False)

In [216]:
paises.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Entity               176 non-null    object 
 1   Latitude             175 non-null    float64
 2   Longitude            175 non-null    float64
 3   landArea             175 non-null    float64
 4   PopulationalDensity  175 non-null    object 
 5   id_pais              176 non-null    int64  
 6   Signatario           176 non-null    bool   
 7   id_continente        176 non-null    int64  
dtypes: bool(1), float64(3), int64(2), object(2)
memory usage: 9.9+ KB


In [217]:
continentes.to_csv("data/continentes.csv", index=False)

In [218]:
continentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_continente  7 non-null      int64 
 1   Continente     7 non-null      object
dtypes: int64(1), object(1)
memory usage: 244.0+ bytes
